<a href="https://colab.research.google.com/github/the-rebooted-coder/Dream-Visualiser/blob/main/MInorPresentationStory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install markovify
!pip install tracery
!pip install spacy==2.3.2
!python -m spacy download en_core_web_sm
!curl -L -O https://raw.githubusercontent.com/the-rebooted-coder/Dream-Visualiser/main/MinorDataset.tsv

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=73cc09910e21b0bb00445dc0a483f588ee9b4c30affbf4500db712bcad584e8e
  Stored in directory: /tmp/pip-ephem-wheel-cache-lmjbbe5t/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2834  100  2834    0     0  14683      0 --:--:-- --:--:-- --:--:-- 14683


In [ ]:
sentences = []
for line in open("MinorDataset.tsv"):
    line = line.strip()
    items = line.split("\t")
    sentences.append(
        {'title': items[0],
         'index': int(items[1]),
         'total': int(items[2]),
         'text': items[3]})

In [ ]:
import random

In [ ]:
random.choice(sentences)

{'index': 4,
 'text': 'Harry finally went after his day with a smile.',
 'title': 'The JUET Cafeteria',
 'total': 15}

## Natural language processing

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
words = []
noun_chunks = []
entities = []
# only use 1000 sentences sampled at random by default; comment out this `for...`
# uncomment the `for...` beneath to use every sentence in the corpus.
for i, sent in enumerate(random.sample(sentences, 24)):
#for i, sent in enumerate(sentences):
    if i % 100 == 0:
        print(i, len(sentences))
    doc = nlp(sent['text'])
    words.extend([w for w in list(doc) if w.is_alpha])
    noun_chunks.extend(list(doc.noun_chunks))
    entities.extend(list(doc.ents))

0 24


Just to make sure it worked, print out ten random words:

In [ ]:
for item in random.sample(words, 10):
    print(item.text)

and
that
out
up
post
The
of
at
different
was


Ten random noun chunks:

In [ ]:
for item in random.sample(noun_chunks, 10):
    print(item.text)

The story
studies
JUET
She
life
a friend
the performance
it
teas
his day


Ten random entities:

In [ ]:
for item in random.sample(entities, 10):
    print(item.text)

JUET University
Sunday
Harry
the day
Jaypee University
JUET
the Nehru Plaza
the Jaypee University Cafetreria's
one
his day


In [ ]:
subjects = [chunk for chunk in noun_chunks if chunk.root.dep_ == 'nsubj']
objects = [chunk for chunk in noun_chunks if chunk.root.dep_ == 'dobj']

In [ ]:
random.sample(subjects, 3)

[the event, who, He]

In [ ]:
random.sample(objects, 5)

[a lot, a friend, the university, any decisions, the university]

In [ ]:
nouns = [w for w in words if w.pos_ == "NOUN"]
verbs = [w for w in words if w.pos_ == "VERB"]
past_tense_verbs = [w for w in words if w.tag_ == 'VBD']
adjs = [w for w in words if w.tag_ == "JJ"]
advs = [w for w in words if w.pos_ == "ADV"]

And now we can print out a random sample of any of these:

In [ ]:
for item in random.sample(nouns, 10): # change "nouns" to "verbs" or "adjs" or "advs" to sample from those lists!
    print(item.text)

professors
friend
dhoklas
work
goodbye
university
morning
story
eras
cafe


In [ ]:
people = [e for e in entities if e.label_ == "PERSON"]
locations = [e for e in entities if e.label_ == "LOC"]
times = [e for e in entities if e.label_ == "TIME"]

And then you can print out a random sample:

In [ ]:
for item in random.sample(people, 3): # change "times" to "people" or "locations" to sample those lists
    print(item.text.strip())

Spandan
Yaseer
Harry


In [ ]:
from collections import Counter
word_count = Counter([w.text for w in words])

In [ ]:
word_count['They']

3

... and also tells us which words are most common:

In [ ]:
word_count.most_common(10)

[('the', 19),
 ('and', 16),
 ('a', 13),
 ('The', 10),
 ('university', 7),
 ('to', 7),
 ('of', 7),
 ('her', 6),
 ('that', 6),
 ('is', 6)]

You can make a counter for any of the other lists we've worked with using the same syntax. Just make up a unique variable name on the left of the `=` sign and put the name of the list you want to count in the brackets to the right (replacing `words`). E.g., to find the most common people:

In [ ]:
people_count = Counter([w.text for w in people])

In [ ]:
people_count.most_common(3)

[('Spandan', 2), ('Harry', 2), ('Yaseer', 1)]

The most common past-tense verbs:

In [ ]:
vbd_count = Counter([w.text for w in past_tense_verbs])

In [ ]:
vbd_count.most_common(12)

[('went', 4),
 ('were', 3),
 ('met', 3),
 ('was', 3),
 ('ended', 2),
 ('joined', 1),
 ('enjoyed', 1),
 ('cheered', 1),
 ('shared', 1),
 ('came', 1),
 ('became', 1),
 ('had', 1)]

In [ ]:
def save_counter_tsv(filename, counter, limit=1000):
    with open(filename, "w") as outfile:
        outfile.write("key\tvalue\n")
        for item, count in counter.most_common():
            outfile.write(item.strip() + "\t" + str(count) + "\n")    

In [ ]:
save_counter_tsv("minor-p2.tsv", word_count, 100)

In [ ]:
import tracery
from tracery.modifiers import base_english

In [ ]:
rules = {
    "subject": [w.text for w in subjects],
    "object": [w.text for w in objects],
    "verb": [w.text for w in past_tense_verbs if w.text not in ('was', 'were', 'went')], # exclude common irregular verbs
    "adj": [w.text for w in adjs],
    "people": [w.text for w in people],
    "loc": [w.text for w in locations],
    "time": [w.text for w in times],
    "origin": "#scene#\n\n[charA:#subject#][charB:#subject#][prop:#object#]#sentences#",
    "scene": "SCENE: #loc#, #time.lowercase#",
    "sentences": [
        "#sentence#\n#sentence#",
        "#sentence#\n#sentence#\n#sentence#",
        "#sentence#\n#sentence#\n#sentence#\n#sentence#"
    ],
    "sentence": [
        "#charA.capitalize# #verb# #prop#.",
        "#charB.capitalize# #verb# #prop#.",
        "#prop.capitalize# became #adj#.",
        "#charA.capitalize# and #charB# greeted each other.",
        "'Did you hear about #object.lowercase#?' said #charA#.",
        "'#subject.capitalize# is #adj#,' said #charB#.",
        "#charA.capitalize# and #charB# #verb# #object#.",
        "#charA.capitalize# and #charB# looked at each other.",
        "#sentence#\n#sentence#"
    ]
}

In [ ]:
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

## Markov chain text generation

In [ ]:
!pip install markovify

In [ ]:
import markovify

In [ ]:
all_text = [item['text'] for item in sentences]

In [ ]:
all_text_gen = markovify.Text(all_text)

In [ ]:
print(all_text_gen.make_sentence())

She also mentions that yes surely she should join the university for her post-graduation.


The `.make_short_sentence()` method allows you to specify a maximum length for the generated sentence:

In [ ]:
print(all_text_gen.make_short_sentence(50))

She has a friend who is interested in meeting her.


In [ ]:
print(all_text_gen.make_short_sentence(40, tries=100))

None


In [ ]:
print(all_text_gen.make_short_sentence(40, test_output=False))

They shared goodbye and never met again.


### Changing the order

In [ ]:
gen_1 = markovify.Text(all_text, state_size=1)
gen_4 = markovify.Text(all_text, state_size=20)

In [ ]:
print("order 1")
print(gen_1.make_sentence(test_output=False))
print()
print("order 4")
print(gen_4.make_sentence(test_output=False))

order 1
The protagonist visits the professors in the Jaypee University Cafetreria's four friends met Harry a while

order 4
Spandan also joined both of them and then they together ended up at the park.


### Changing the level

In [ ]:
class SentencesByChar(markovify.Text):
    def word_split(self, sentence):
        return list(sentence)
    def word_join(self, words):
        return "".join(words)

In [ ]:
gen_char = SentencesByChar(all_text, state_size=7)

In [ ]:
print(gen_char.make_sentence(test_output=False))

Harry finally went ahead to find Spandan also joined both of the protagonist, a students of Jaypee University has been under dispute because some student from JUET, who is interested in meeting her.


### Thinking about structure


In [ ]:
beginnings = [line['text'] for line in sentences if line['index'] == 0]

In [ ]:
random.sample(beginnings, 1)

['The story is about the tuck  JUET University. The story starts with the protagonist, a student from JUET, who is interested in joining the university for her post-graduation.']

And endings are sentences that come last in the plot (i.e., their index is one less than the total number of sentences):

In [ ]:
endings = [line['text'] for line in sentences if line['index'] == line['total'] -1]

And "middles" are anything in between:

In [ ]:
middles = [line['text'] for line in sentences if 0 < line['index'] < line['total'] - 1]

In [ ]:
random.sample(middles, 5)

['On a morning sunrise Manvendra went near the lake',
 'On this night, the university would play music from different cultures and different eras.',
 'She meets one of her professors who mentions that yes surely she should join the university as these are myths',
 'This year Yaseer came to perform some of his best songs',
 'The preparations are being done by the students of Jaypee University.']

In [ ]:
beginning_gen = markovify.Text(beginnings)
middle_gen = markovify.Text(middles)

Now you can generate tiny narratives by producing a beginning sentence, a middle sentence, and an ending sentence:

In [ ]:
print(beginning_gen.make_short_sentence(100))
print(middle_gen.make_short_sentence(100))

None
The protagonist decides to meet some of his best songs


### Combining models

In [ ]:
combo = markovify.combine([beginning_gen, middle_gen], [2, 10])

In [ ]:
print(combo.make_short_sentence(120))

They went together to the Nehru Plaza and sat over there and is also interested in meeting her.


## Prepping the corpus for fine-tuning a large language model

In [ ]:
out = []
last_title = None
for sent in sentences[:3]:
    if sent['title'] != last_title:
        out.append("")
        out.append("[BEGIN STORY]")
        out.append(sent['title'])
        out.append("")
        last_title = sent['title']
    out.append(sent['text'])

In [ ]:
out[:10]

['',
 '[BEGIN STORY]',
 '\ufeffThe Disputed Tuck Shop',
 '',
 'The story is about the tuck  JUET University. The story starts with the protagonist, a student from JUET, who is interested in joining the university for her post-graduation.',
 '',
 '[BEGIN STORY]',
 'The Disputed Tuck Shop',
 '',
 'She meets Varun there. Then they go together to meet Spandan, and never meet again.']

In [ ]:
with open("story_training.txt", "w") as fh:
    fh.write("\n".join(out))